In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("datascience.csv", encoding='gb18030')

In [5]:
df.head()

,title,author,content
0,大数据产业迎政策暖风 最新大数据概念股一览,财经热点扒客,大数据产业发展受到国家重视，而大数据已经上升为国家战略，未来发展前景很广阔。大数据产业“十三...
1,Google发布机器学习平台Tensorflow游乐场～带你一起玩神经网络！,硅谷周边,点击上方“硅谷周边”关注我，收到最新的文章哦！昨天，Google发布了Tensorflow游...
2,李克强：中国大数据和云计算产业是开放的,苏州高新区金融办,国务院总理李克强当地时间20日上午在纽约下榻饭店同美国经济、金融、智库、媒体等各界人士座谈，...
3,全峰集团持续挖掘大数据,快递物流网,2016年，全峰集团持续挖掘大数据、云计算、“互联网+”等前沿技术和物流快递的融合，并通过优...
4,第366期【微理工】贵州理工学院召开大数据分析与应用专题分享会,贵州理工学院,贵州理工学院召开大数据分析与应用专题分享会 借“创响中国”贵安站巡回接力活动暨2016贵安大...


In [9]:
df.shape

(1024, 3)

In [10]:
import jieba

In [11]:
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [12]:
df["content_cutted"] = df.content.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/rq/1zvf7k596jb3fdp01c1nrvxr0000gp/T/jieba.cache
Loading model cost 3.179 seconds.
Prefix dict has been built succesfully.


In [13]:
df.content_cutted.head()
#执行时间比较长，需等待

0    大 数据 产业 发展 受到 国家 重视 ， 而 大 数据 已经 上升 为 国家 战略 ， 未...
1    点击 上方 “ 硅谷 周边 ” 关注 我 ， 收到 最新 的 文章 哦 ！ 昨天 ， Goo...
2    国务院 总理 李克强 当地 时间 20 日 上午 在 纽约 下榻 饭店 同 美国 经济 、 ...
3    2016 年 ， 全峰 集团 持续 挖掘 大 数据 、 云 计算 、 “ 互联网 + ” 等...
4    贵州 理工学院 召开 大 数据分析 与 应用 专题 分享 会   借 “ 创响 中国 ” 贵...
Name: content_cutted, dtype: object

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [15]:
n_features = 1000

In [31]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words=("这个","就是","的","www","com","http","可能","一些","什么","他们","没有","自己","如果","很多","不是","但是"
                                            ,"现在","这样","因为","已经","时候","还是","可是",),
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

In [32]:
from sklearn.decomposition import LatentDirichletAllocation

In [33]:
n_topics = 5
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [34]:
lda.fit(tf)
#执行时间比较长，需等待

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_jobs=1, n_topics=5, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [35]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [36]:
n_top_words = 20

In [37]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
企业 中国 平台 服务 行业 互联网 金融 管理 公司 创新 数据分析 2016 城市 产业 市场 实现 国家 用户 建设 业务
Topic #1:
用户 产品 数据分析 所以 时间 发现 工作 孩子 非常 如何 对于 公司 研究 还是 能够 预测 不同 价值 比如 那么
Topic #2:
人工智能 领域 智能 机器人 公司 人类 学习 机器 未来 市场 投资 医疗 目前 识别 研究 深度 服务 企业 工业 方面
Topic #3:
系统 数据库 使用 电子 存储 采集 处理 或者 支持 数据仓库 检索 查询 hadoop 设备 管理 实现 业务 收集 功能 过程
Topic #4:
学习 算法 模型 机器 方法 使用 神经网络 可视化 深度 特征 训练 the and 分类 计算 函数 不同 data 基于 网络



In [38]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      27.620445        1       1  0.010411 -0.036866
0      24.696045        1       2 -0.164652  0.072342
4      19.686158        1       3  0.161582 -0.093408
2      15.094597        1       4 -0.126795 -0.083118
3      12.902755        1       5  0.119454  0.141051, topic_info=     Category         Freq  Term        Total  loglift  logprob
term                                                           
130   Default  2779.000000  人工智能  2779.000000  30.0000  30.0000
369   Default  4218.000000    学习  4218.000000  29.0000  29.0000
618   Default  1112.000000   机器人  1112.000000  28.0000  28.0000
781   Default  1855.000000    算法  1855.000000  27.0000  27.0000
587   Default  1453.000000    智能  1453.000000  26.0000  26.0000
617   Default  2193.000000    机器  2193.000000  25.0000  25.0000
154   Default  3211.000000    企业  3211.000000  24.0000  24.0000
647   Default  1781.000000    模型  1781.000000  23.0000  23.0000
992   Default  2006.000000    领域  2006.000000  22.0000  22.0000
198   Default  2436.000000    公司  2436.000000  21.0000  21.0000
787   Default  1997.000000    系统  1997.000000  20.0000  20.0000
757   Default   916.000000  神经网络   916.000000  19.0000  19.0000
720   Default   722.000000    电子   722.000000  18.0000  18.0000
558   Default   833.000000   数据库   833.000000  17.0000  17.0000
171   Default  2233.000000    使用  2233.000000  16.0000  16.0000
133   Default  1020.000000    人类  1020.000000  15.0000  15.0000
367   Default   739.000000    存储   739.000000  14.0000  14.0000
575   Default  1531.000000    方法  1531.000000  13.0000  13.0000
956   Default   752.000000    采集   752.000000  12.0000  12.0000
679   Default  1343.000000    深度  1343.000000  11.0000  11.0000
97    Default  1764.000000    中国  1764.000000  10.0000  10.0000
961   Default   992.000000    金融   992.000000   9.0000   9.0000
296   Default  1020.000000   可视化  1020.000000   8.0000   8.0000
782   Default  1361.000000    管理  1361.000000   7.0000   7.0000
117   Default  1710.000000   互联网  1710.000000   6.0000   6.0000
847   Default  1584.000000    行业  1584.000000   5.0000   5.0000
877   Default   657.000000    训练   657.000000   4.0000   4.0000
609   Default  1744.000000    服务  1744.000000   3.0000   3.0000
52    Default   586.000000   the   586.000000   2.0000   2.0000
30    Default   567.000000   and   567.000000   1.0000   1.0000
...       ...          ...   ...          ...      ...      ...
640    Topic5   102.828556    检查   147.161469   1.6893  -6.2174
718    Topic5   179.572905    申请   260.088882   1.6773  -5.6599
262    Topic5   103.733886    医院   150.388653   1.6763  -6.2086
761    Topic5   100.735035    科研   146.937930   1.6702  -6.2380
235    Topic5   159.451284   分布式   236.217360   1.6547  -5.7787
626    Topic5   254.355827    架构   405.917572   1.5803  -5.3117
541    Topic5   334.673419    收集   561.722587   1.5299  -5.0373
787    Topic5   950.524869    系统  1997.250705   1.3052  -3.9935
384    Topic5   262.085649    实时   432.260669   1.5474  -5.2818
881    Topic5   359.738997    设备   650.330279   1.4556  -4.9651
122    Topic5   226.272130    交通   362.585985   1.5762  -5.4287
878    Topic5   276.215574    记录   487.304942   1.4800  -5.2293
348    Topic5   505.013872    处理  1277.418768   1.1197  -4.6259
486    Topic5   464.167167    或者  1129.076392   1.1588  -4.7102
171    Topic5   685.313527    使用  2233.233813   0.8664  -4.3206
537    Topic5   397.081133    支持   924.018221   1.2031  -4.8663
248    Topic5   318.117687    功能   658.747754   1.3198  -5.0881
583    Topic5   286.951209    是否   679.328316   1.1859  -5.1912
91     Topic5   338.036697    业务  1122.836116   0.8473  -5.0273
782    Topic5   353.702157    管理  1361.656073   0.6997  -4.9820
853    Topic5   266.039163    要求   582.483671   1.2641  -5.2668
385    Topic5   339.560451    实现  1645.065059   0.4698  -5.0228
924    Topic5   309.642924    过程  1130.512782   0.7527  -5.

In [39]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
#运算时间长，需等待
lda.fit(tf)

print_top_words(lda, tf_feature_names, n_top_words)
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

Topic #0:
企业 管理 服务 创新 平台 建设 政府 产业 业务 实现 资源 建立 能力 国家 开放 互联网 信息化 安全 项目 相关
Topic #1:
工作 所以 非常 研究 能够 发现 还是 如何 这种 知道 那么 时间 比如 应该 所有 情况 重要 对于 或者 看到
Topic #2:
人工智能 智能 领域 机器人 公司 学习 人类 机器 医疗 未来 识别 研究 目前 服务 语音 投资 科技 系统 工业 ai
Topic #3:
电子 采集 系统 或者 应当 检索 案件 设备 收集 是否 保护 规定 信用卡 记录 提取 法律 专利 申请 相关 法院
Topic #4:
学习 机器 深度 神经网络 算法 模型 the and 训练 of 网络 方法 to data in 图片 使用 learning for 系统
Topic #5:
孩子 学习 教育 学生 课程 工作 数据分析 交通 老师 经验 数据挖掘 培训 专业 教授 学校 时间 10 阅读 学院 报名
Topic #6:
中国 2016 增长 市场 2015 城市 10 大众 人口 其中 同比 美国 行业 投资 亿元 20 全国 关注 12 30
Topic #7:
可视化 使用 数据库 存储 工具 系统 处理 hadoop 用户 图表 支持 数据仓库 查询 开发 基于 设计 sql 平台 计算 架构
Topic #8:
用户 互联网 产品 金融 公司 企业 客户 行业 平台 营销 服务 数据分析 市场 消费者 价值 广告 移动 网络 银行 电商
Topic #9:
模型 算法 方法 特征 分类 预测 使用 样本 数据挖掘 函数 计算 不同 结果 变量 回归 过程 维度 规则 关联 关系



PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      19.135399        1       1  0.054366 -0.055820
8      12.711950        1       2 -0.119021  0.004219
0      11.860002        1       3 -0.156902  0.096884
2      10.934179        1       4 -0.089528 -0.084869
9       9.969713        1       5  0.184375 -0.041003
7       9.425070        1       6  0.091109  0.018236
4       8.424612        1       7  0.161133 -0.164134
6       8.248615        1       8 -0.162597 -0.012446
5       5.206376        1       9 -0.117469 -0.016687
3       4.084083        1      10  0.154534  0.255620, topic_info=     Category         Freq  Term        Total  loglift  logprob
term                                                           
369   Default  4206.000000    学习  4206.000000  30.0000  30.0000
130   Default  2778.000000  人工智能  2778.000000  29.0000  29.0000
154   Default  3206.000000    企业  3206.000000  28.0000  28.0000
617   Default  2182.000000    机器  2182.000000  27.0000  27.0000
781   Default  1848.000000    算法  1848.000000  26.0000  26.0000
647   Default  1776.000000    模型  1776.000000  25.0000  25.0000
618   Default  1112.000000   机器人  1112.000000  24.0000  24.0000
715   Default  2498.000000    用户  2498.000000  23.0000  23.0000
720   Default   714.000000    电子   714.000000  22.0000  22.0000
757   Default   907.000000  神经网络   907.000000  21.0000  21.0000
296   Default  1028.000000   可视化  1028.000000  20.0000  20.0000
97    Default  1760.000000    中国  1760.000000  19.0000  19.0000
679   Default  1336.000000    深度  1336.000000  18.0000  18.0000
587   Default  1454.000000    智能  1454.000000  17.0000  17.0000
373   Default   633.000000    孩子   633.000000  16.0000  16.0000
956   Default   747.000000    采集   747.000000  15.0000  15.0000
550   Default   648.000000    教育   648.000000  14.0000  14.0000
117   Default  1714.000000   互联网  1714.000000  13.0000  13.0000
992   Default  2005.000000    领域  2005.000000  12.0000  12.0000
486   Default  1126.000000    或者  1126.000000  11.0000  11.0000
171   Default  2232.000000    使用  2232.000000  10.0000  10.0000
961   Default   997.000000    金融   997.000000   9.0000   9.0000
420   Default  1402.000000    市场  1402.000000   8.0000   8.0000
198   Default  2439.000000    公司  2439.000000   7.0000   7.0000
52    Default   580.000000   the   580.000000   6.0000   6.0000
787   Default  1991.000000    系统  1991.000000   5.0000   5.0000
575   Default  1528.000000    方法  1528.000000   4.0000   4.0000
30    Default   561.000000   and   561.000000   3.0000   3.0000
346   Default   805.000000    增长   805.000000   2.0000   2.0000
782   Default  1358.000000    管理  1358.000000   1.0000   1.0000
...       ...          ...   ...          ...      ...      ...
718   Topic10   175.056060    申请   258.613850   2.8078  -4.5350
262   Topic10    95.955088    医院   149.290421   2.7561  -5.1363
270   Topic10   117.845722    原始   186.327644   2.7399  -4.9308
176   Topic10   217.472166    保护   346.175904   2.7332  -4.3181
640   Topic10    88.588807    检查   146.152225   2.6974  -5.2161
532   Topic10   189.521214    提取   314.716867   2.6909  -4.4556
956   Topic10   415.271578    采集   747.087257   2.6108  -3.6712
761   Topic10    79.606334    科研   146.177205   2.5903  -5.3230
643   Topic10    99.129567    检验   188.443879   2.5557  -5.1037
475   Topic10    88.081201    意见   179.281633   2.4874  -5.2219
180   Topic10   132.307396    信号   285.463350   2.4291  -4.8150
773   Topic10    84.640360    符合   184.449362   2.4191  -5.2617
199   Topic10    88.590182    公开   207.913034   2.3450  -5.2161
541   Topic10   231.942751    收集   559.417427   2.3177  -4.2536
878   Topic10   194.145834    记录   485.201527   2.2821  -4.4315
245   Topic10    72.298528    制作   183.912087   2.2644  -5.4193
881   Topic10   244.228110    设备   646.869932   2.2240  -4.2020
179   Topic10    66.766355    保障   201.489976   2.0935  -5.4989
604   Topic10    85.978667    有关   259.626329